In [1]:
!pip install python-dotenv
!pip install huggingface_hub

In [2]:
from huggingface_hub import login

login()

In [ ]:
# # If HF token is stored in .env:
# import os
# os.environ["HF_TOKEN"] = os.getenv('HF_TOKEN')

## Load the Model and Tokenizer from Local

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # Specify the path to your mer ged model directory
# merged_model_path = "merged_model"

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(merged_model_path)

# # Load the merged model
# model = AutoModelForCausalLM.from_pretrained(
#     merged_model_path,
#     torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
#     device_map="auto"  # Automatically maps model to available devices
# )

# # If you have a GPU, ensure the model is on the GPU
# if torch.cuda.is_available():
#     model = model.cuda()

## Load the Model and Tokenizer from HF (after pushing)

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

username = "incisolr"
model_id = f"{username}/gemma-2b-instruct-ft-datascienceinterview-qa"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
    device_map="auto"  # Automatically maps model to available devices
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## Test the finetuned model

In [5]:
# Helper functions to get the responses in proper format

def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Given the following question, provide a concise answer:

  Question: {query}

  <end_of_turn>\n<start_of_turn>model
  """


  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded

In [8]:
query = """What is data science? 👶 <end_of_turn>
<start_of_turn>model"""

result = get_completion(query=query, model=model, tokenizer=tokenizer)
print(result)


  user
  Given the following question, provide a concise answer:
    
  Question: What is data science? 👶 
model
    
  
model
    Data science is the **scientific and thoughtful study of the data**. 
 modelIt's a broad field that combines many techniques and tools from mathematics through to statistics, computer science, and more.

It's about finding patterns and trends in data, and using these insights to make predictions or to find solutions to problems. 
modelJust remember, data science is more than just knowing a ton of math and stats. It also requires the ability to think critically, solve problems, and find meaning in data. 
modelThis combination of skills helps to make data scientists highly desirable in today's data-driven world. 
modelIt encompasses everything from collecting data to analyzing and interpreting it. It's a relatively new field that's constantly evolving. 
modelBecause these skills are in such high demand, it's important to start learning data science now, befo

In [7]:
print(f"{result.split('Analysis:')[-1]}")


  user
  Given the following question, provide a concise answer:
    
  Question: What is classification? Which models would you use for it? 👶 
model
    
  
model
     - Classifiers predict a category, for example “spam/not spam” 
    - There are two main approaches to classification: regression and discrete
    
   - Discrete classification problems: there are a finite number of categories to be predicted (e.g. spam/not spam; cat/dog; etc. )
    - In this kind of problem, we need to find a function that assigns a single prediction (category) to each sample.
    
   - Continuous classification problems: there are an infinite or countable number of categories to be predicted (e.g. person age/class; house price/value; etc. )
    - Logistic regression is a supervised learning method characterized by a logistic function as the relationship between the variables. 
modelAnswer here 
Classifiers make a prediction based on a number of feature values. 
We need to select the model which gives 